In [9]:
import pandas as pd
import numpy as np
import pickle
import math
import warnings
# !pip install yfinance
import yfinance as yf
"""Evaluate trading strategy performance over time interval."""
from typing import List, Tuple, Dict
# from datetime import datetime
import datetime
warnings.filterwarnings("ignore")



In [2]:
train_df = pd.read_csv('8k_data_labels.csv')

In [3]:
def load_historical_data(filename):
    with open(filename, 'rb') as handle:
        tikr_dict = pickle.load(handle)
        return tikr_dict
TIKRS_dat = load_historical_data('TIKR_DATA.pickle')

In [32]:
class StockSimulation:
    def __init__(self, cash=1000000, tikrs = ['aapl','msft'], historical_data=TIKRS_dat):
        self.cash = cash
        self.tikrs = tikrs
        self.portfolio = {}
        for tikr in tikrs:
          self.portfolio[tikr] = 0
        self.active_log = []
        self.transaction_log = []
        self.transaction_cost = 0.01
        self.historical_data = TIKRS_dat
        
    def get_price(self, tikr, date):
        start = date
        if type(start) is str:
          start = datetime.datetime.strptime(start, '%Y%m%d')
          
        date = start.strftime('%Y-%m-%d')

        company_df = self.historical_data[tikr]
        return company_df[company_df['Date'] == date].iloc[0]['Open']
#         return yf.download(
#             tikr, 
#             start=start,
#             interval= '1d', 
#             progress= False, ).loc[date]['Adj Close']



    def buy(self, tikr, date, allocated_money):
        # check if there is enough cash to buy
        if self.cash < allocated_money:
            print("Not enough cash to buy")
            return

        # calculate number of shares to buy
        price = self.get_price(tikr, date)
        shares = int(allocated_money / price)

        # update portfolio and cash balance
        if tikr in self.active_log:
            self.portfolio[tikr] += shares
        else:
            self.portfolio[tikr] = shares
            self.active_log.append(tikr)
        # minus the cost of stock
        stock_cost = shares * price
        # minus trasaction cost
        self.cash -= stock_cost * (1 + self.transaction_cost)


        # log transaction
        self.transaction_log.append({
            "type": "buy",
            "tikr": tikr,
            "date": date,
            "shares": shares,
            "price": price,
            "amount": stock_cost ,
            "transaction_cost": stock_cost * self.transaction_cost,
            "balance": self.active_balance(date)
        })

    def sell(self, tikr, date, allocated_money):
        # check if there are enough shares to sell
        if tikr not in self.portfolio:
            print("No shares of {} in portfolio".format(tikr))
            return


        # calculate number of shares to sell
        price = self.get_price(tikr, date)
        shares = int(allocated_money / price)


        if self.portfolio[tikr] < shares:
            print("Not enough shares of {} to sell".format(tikr))
            return

        # update portfolio and cash balance
        self.portfolio[tikr] -= shares
        if self.portfolio[tikr] == 0:
            self.active_log.remove(tikr)

        # plus the earning
        stock_cost = shares * price
        self.cash += stock_cost
        # minus trasaction cost
        self.cash -= stock_cost *  self.transaction_cost


        # log transaction
        self.transaction_log.append({
            "type": "sell",
            "tikr": tikr,
            "date": date,
            "shares": shares,
            "price": price,
            "amount": stock_cost ,
            "transaction_cost": stock_cost * self.transaction_cost,
            "balance": self.active_balance(date)
        })

    #TODO
    def get_next_trading_date(self,tikr, date):
        return None

    def rebalance(self,percentage,date):
      buy = []
      for tikr, percent in zip(self.tikrs, percentage):
        price = self.get_price(tikr, date)
        expected_value = self.active_balance(date) * percent
        current_value = self.portfolio[tikr] * price

        allocated_money = expected_value - current_value

        if allocated_money > 0:
          self.sell(tikr, date, allocated_money)
        else:
          buy += [(tikr, allocated_money)]

      for tikr, allocated_money in buy:
        self.buy(tikr, date, allocated_money)




    def active_balance(self, date):
        balance = self.cash
        for tikr in self.active_log:
            price = self.get_price(tikr, date)
            balance += self.portfolio[tikr] * price
        return balance

    def transaction_summary(self):
        for txn in self.transaction_log:
            print("{} {} {} shares of {} at ${:.2f} for ${:.2f} (transaction cost: ${:.2f}), balance: ${:.2f}".format(
                txn['date'], txn['type'], txn['shares'], txn['tikr'], txn['price'], txn['amount'], txn['transaction_cost'], txn['balance']))


In [31]:



""" All trading strategies should have standardized input to work in automated
    downstream testing."""


def trading_strategy(
        predictions: List[Tuple(float, float, float)],
        company_list: List[str]
            ) -> Dict[datetime, List[float]]:
    """Calculate portfolio holdings at time intervals given 8-K labels."""

    strategy = dict()

    return strategy


def get_strategy_annual_return(
        strategy: Dict[datetime, List[float]],
        company_list: List[str],
        end_date: datetime,
        starting_balance: int = 1e7,
        start_date: datetime = '20000101') -> float:
    """
    Calculate annual return over time period.
    
    Parameters
    ----------
    strategy: Dict[datetime, List[float]]
        A dictionary of portfolio rebalance dates associated with portfolio
        allocation percentages. The portfolio allocation percentages share the
        same indexing as company_list.
    company_list: List[str]
        The companies invested into by the trading strategy percent allocation.
    start_date: datetime, str
        Accepted format "year_month_day", or datetime object. The first day of
        trading.
    end_date: datetime, str
        Accepted format "year_month_day", or datetime object. The final day of
        trading used to determine net worth and annualized return.
    starting_balance: int
        The initial amount of money invested.
    Returns
    -------
    Annualized Return: float
        The annualized return rate, where 0% return indicates 1.00.
    """

    if type(start_date) is str:
        start_date = datetime.strptime(start_date, "%Y%m%d")
    if type(end_date) is str:
        end_date = datetime.strptime(end_date, "%Y%m%d")

    for date, value in strategy.items()

    # At each date, rebalance current networth to be distributed
    # percentage-wise between companies in portfolio_allocations

    return 1.00

SyntaxError: expected ':' (186977707.py, line 52)

In [36]:
s = StockSimulation()
s.get_price('msft', '20000110')
s.buy('msft', '20000110', 10000)
s.sell('msft', '20000110', 10000)
s.active_balance('20000110')


999800.3500000001

In [37]:
s.cash 

999800.3500000001

In [38]:
s.transaction_summary()

20000110 buy 176 shares of msft at $56.72 for $9982.50 (transaction cost: $99.83), balance: $999900.18
20000110 sell 176 shares of msft at $56.72 for $9982.50 (transaction cost: $99.83), balance: $999800.35


In [39]:
data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")


[*********************100%***********************]  2 of 2 completed


In [27]:
data

Adj Close                  Close                   High  \
                 AAPL         SPY       AAPL         SPY       AAPL   
Date                                                                  
2017-01-03  27.133326  202.856552  29.037500  225.240005  29.082500   
2017-01-04  27.102957  204.063339  29.004999  226.580002  29.127501   
2017-01-05  27.240786  203.901276  29.152500  226.399994  29.215000   
2017-01-06  27.544476  204.630737  29.477501  227.210007  29.540001   
2017-01-09  27.796770  203.955292  29.747499  226.460007  29.857500   
...               ...         ...        ...         ...        ...   
2017-04-24  33.700623  214.530182  35.910000  237.169998  35.987499   
2017-04-25  33.909443  215.778519  36.132500  238.550003  36.224998   
2017-04-26  33.710014  215.642822  35.919998  238.399994  36.150002   
2017-04-27  33.735817  215.823700  35.947498  238.600006  36.040001   
2017-04-28  33.702972  215.353363  35.912498  238.080002  36.075001   

                              Low                   Open              \
                   SPY       AAPL         SPY       AAPL         SPY   
Date                                                                   
2017-01-03  225.830002  28.690001  223.880005  28.950001  225.039993   
2017-01-04  226.750000  28.937500  225.610001  28.962500  225.619995   
2017-01-05  226.580002  28.952499  225.479996  28.980000  226.270004   
2017-01-06  227.750000  29.117500  225.899994  29.195000  226.529999   
2017-01-09  227.070007  29.485001  226.419998  29.487499  226.910004   
...                ...        ...         ...        ...         ...   
2017-04-24  237.410004  35.794998  234.559998  35.875000  237.179993   
2017-04-25  238.949997  35.967499  237.809998  35.977501  237.910004   
2017-04-26  239.529999  35.845001  238.350006  36.117500  238.509995   
2017-04-27  238.949997  35.827499  237.979996  35.980000  238.770004   
2017-04-28  238.929993  35.817501  237.929993  36.022499  238.899994   

               Volume             
                 AAPL        SPY  
Date                              
2017-01-03  115127600   91366500  
2017-01-04   84472400   78744400  
2017-01-05   88774400   78379000  
2017-01-06  127007600   71559900  
2017-01-09  134247600   46939700  
...               ...        ...  
2017-04-24   68537200  119209900  
2017-04-25   75486000   76698300  
2017-04-26   80164800   84702500  
2017-04-27   56985200   57410300  
2017-04-28   83441600   63532800  

[81 rows x 12 columns]